In [8]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import losses, activations, models
from keras.layers import Dense, InputLayer,BatchNormalization, Dropout, Conv2D, Flatten, Activation

In [2]:
mnist = keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
# Normalize the input image so that each pixel value is between 0 to 1.
x_train = x_train / 255.0
x_test = x_test / 255.0

In [9]:
def get_model():
       inputs = keras.Input(shape=(28, 28, 1))
       x = Conv2D(32, kernel_size=(3, 3),
                     strides=(2, 2), padding="same",
                     use_bias=False)(inputs)
       x = BatchNormalization()(x)
       x = Activation("relu")(x)
       x = Conv2D(64, kernel_size=(3, 3),
                     strides=(2, 2), padding="same",
                     use_bias=False)(x)
       x = BatchNormalization()(x)
       x = Activation("relu")(x)
       x = Flatten()(x)
       x = Dropout(0.5)(x)
       outputs = Dense(10, activation="softmax")(x)

       model = keras.Model(inputs=inputs, outputs=outputs)
       
       opt = tf.keras.optimizers.Adam(0.001)
       model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])
       return model


In [10]:
model_given = get_model()
model_given.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 14, 14, 32)        288       
                                                                 
 batch_normalization (BatchN  (None, 14, 14, 32)       128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 14, 14, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 64)          18432     
                                                                 
 batch_normalization_1 (Batc  (None, 7, 7, 64)         256       
 hNormalization)                                             

In [6]:
# Function to obtain the output from each layer of the model (To be used for the repeat task type)
output_size = {}
def keras_model_size(model):
    prefix_name = model.name + "_"
    for layer in model.layers:
        if isinstance(layer, (tf.keras.Model,models.Sequential)):
            keras_model_size(layer)
        else:
          out_shape = layer.output_shape
        output_size[prefix_name + layer.name] = out_shape
       
          
    return output_size


In [7]:
keras_model_size(model_given)

{'model_input_1': [(None, 28, 28, 1)],
 'model_conv2d': (None, 14, 14, 32),
 'model_batch_normalization': (None, 14, 14, 32),
 'model_activation': (None, 14, 14, 32),
 'model_conv2d_1': (None, 7, 7, 64),
 'model_batch_normalization_1': (None, 7, 7, 64),
 'model_activation_1': (None, 7, 7, 64),
 'model_flatten': (None, 3136),
 'model_dropout': (None, 3136),
 'model_dense': (None, 10)}

In [8]:
# Obtains the output in bits, the memory requirements of each layer

def keras_model_output(model):
    output_dict = {}
    default_dtype = tf.keras.backend.floatx()
    prefix_name = model.name + "_"
    for layer in model.layers:
        if isinstance(layer, (tf.keras.Model,models.Sequential)):
            keras_model_output(layer)
        else:
            no_of_neurons = tf.as_dtype(layer.dtype or default_dtype).size # dtype is float 32 and the size here is 4 bytes(32 bits)
            out_shape = layer.output_shape
            if isinstance(out_shape, list):
                for s in out_shape[0]:
                    if s is None:
                        continue
                    no_of_neurons *= s # Output obtained here is in 4 Bytes 
            else:
                for s in out_shape:
                    if s is None:
                        continue
                    no_of_neurons *= s # Output obtained here is in 4 Bytes 
            no_of_neurons *= 8 # Output obtained in bits
        output_dict[prefix_name + layer.name] = no_of_neurons
       
          
    return output_dict

In [9]:
keras_model_output(model_given)

{'model_input_1': 25088,
 'model_conv2d': 200704,
 'model_batch_normalization': 200704,
 'model_activation': 200704,
 'model_conv2d_1': 100352,
 'model_batch_normalization_1': 100352,
 'model_activation_1': 100352,
 'model_flatten': 100352,
 'model_dropout': 100352,
 'model_dense': 320}

In [10]:
# EFFECTIVE DATA COMMUNICATION PROBABILITY

# Function to print the effective data communicated 
def data_summary(model,x_test,y_test):

  # Creating dictionary for storing the percent of data communicated eliminating the sparse data
  effective_probability = {}
  
  # Function to add communicated data to the metrics
  def add_metrics(m2):
    "Annotates recursiverly a (non-compiled) model with metrics that calculate probability of non-zero activity"
    prefix_name = m2.name + "_"

    for ll in m2.layers:
        if is_hier_layer(ll):
            add_metrics(ll)
        else:
            my_metric = tf.reduce_sum( tf.math.count_nonzero(ll.output, dtype=tf.int32) / tf.size( ll.output)  )
            name=prefix_name + ll.name
            m2.add_metric(my_metric, name=name)
        
    return

  # Function to check if a layer is model or not
  def is_hier_layer(layer):
      "Finds if layer is actually a model instead of a single layer"
      return type(layer) in [models.Sequential, keras.Model]

  # Clone the model. Note that it is not compiled!
  m2 = keras.models.clone_model(model)
  add_metrics(m2)
  
  # Compiling the cloned model and take the inference
  m2.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  m2.evaluate(x_test[0:100], y_test[0:100]) 
  
  # print('%40s | %20s' % ('Metrics and Layer Name', 'Effective data communicated'))
  # print("*" * 150)
  for l in m2.metrics:
    # print('%40s | %5.4f' % (l.name, l.result().numpy()))
    if l.name!="loss" and l.name!="accuracy" :
      effective_probability[l.name] = l.result().numpy()      

  return effective_probability


In [11]:
data_summary(model_given, x_train, y_train)

4/4 [==============================] - 1s 15ms/step - loss: 2.3009 - accuracy: 0.1200 - model_input_1: 0.4867 - model_conv2d: 0.6380 - model_batch_normalization: 0.6380 - model_activation: 0.3420 - model_conv2d_1: 0.7921 - model_batch_normalization_1: 0.7921 - model_activation_1: 0.4148 - model_flatten: 0.4148 - model_dropout: 0.4148 - model_dense: 1.0000


{'model_input_1': 0.4867267219387755,
 'model_conv2d': 0.6379942602040817,
 'model_batch_normalization': 0.6379942602040817,
 'model_activation': 0.3419986647002551,
 'model_conv2d_1': 0.7920918367346939,
 'model_batch_normalization_1': 0.7920918367346939,
 'model_activation_1': 0.4147824657206633,
 'model_flatten': 0.4147824657206633,
 'model_dropout': 0.4147824657206633,
 'model_dense': 1.0}

In [1]:
# to count the number of tasks required:

def layer_count(model):
    count = 0
    for l in model.layers:
        if isinstance(l, (tf.keras.Model,models.Sequential)):
            for ll in l.layers:
                count = count + 1
        else:
            count = count + 1
    return (f'tasks_number = {count+1}')

In [11]:
layer_count(model_given)

'tasks_number = 11'